# How do I send an API request?

1. specifically how can i figure out the API request when there is no documentation?
2. How does Domo handle authentication?

**Domo uses "username and password" to generate an token which is then used to authenticate most API requests that get sent in the user interface.**

1. Go to Data > Accounts to view a list of all accounts you have access to.

2. Monitor network traffic (different for each browser -- Inspect > Network on Chrome) to see what API request is being sent to retrieve a list of auth objects.

3. Find the API request.  Filter for "providers".  Examine the URL, headers, request Method (GET, PUT, POST, DELETE) and body<br>

   usually you can right click the query and copy paste into raw text
   
   below, the CURL request had a few unnecessary attributes removed, but the cookie is handling authentication.

## convert an API request into a function
- We will use the requests library to send an API request to the providers endpoint.
- In the above sample, authentication is handled by the cookie.  in a later step we will show how to handle authentication in Domo.
- Functions allow us to call snippets of code multiple times.
Using functions improves legibility, recycle-ability, and maitainability of code


In [1]:
import requests

DOMO_INSTANCE = 'domo-community' ## it's common practice to declare constants using all caps.

def get_accounts(domo_instance):
    # v1 does not accept authentication and therefore will fail
    
    url = f'https://{domo_instance}.domo.com/api/data/v2/datasources/providers'
    
    return requests.request(method = "GET",
                            url = url)

res = get_accounts(domo_instance = DOMO_INSTANCE)
res

<Response [401]>

### debugging what happened

DEVELOPER_TIP
- get comfortable googling "example of sending an API request with the requests library"
- use the type() function to see the data type of the response

the get_accounts function returns a class "Response" from the requests.request library
the Response class has attributes like status_code and a method json() which allows us to view the response as a json object

In [2]:
print(type(res))
print(res.status_code)
print(res.json())

# from the error message it's clear we need to pass authentication with our API request

<class 'requests.models.Response'>
401
{'status': 401, 'statusReason': 'Unauthorized', 'path': '/api/data/v2/datasources/providers', 'message': 'Full authentication is required to access this resource', 'toe': 'NYPSO7AJYW-7MPCT-WTWVN'}


## Use Full Auth to handle Domo API Authentication

- "full authentication" is not the same as client_id and secret authentication which can be retrieved from developer.domo.com

- client_id and secret auth (we will call developer_token authentication moving forward) only applies to a small handful of APIs documented under developer.domo.com

- "full authentication actually has two flavors.<br>
    - username and password authentication (session_token which has a short expiration date)
    - access_token authentication (Admin > Security > Access Token) which tends to have a longer expiration date, we will cover this later


DEVELOPER TIPS:
- document the type we expect to receive for each input and output variable (str)
- provide a document string (denoted in triple quotes) to describe what the function does
- notice the use of logic to test for success

In [3]:
def get_full_auth(
    domo_instance: str,
    domo_username: str,
    domo_password: str
) -> str:  # returns a session token
    """use username and password to generate an access token"""

    url = f"https://{domo_instance}.domo.com/api/content/v2/authentication"

    body = {
        "method": "password",
        "emailAddress": domo_username,
        "password": domo_password,
    }

    res = requests.request(method="POST", url=url, json=body)
    data = res.json()

    token = data.get("sessionToken")
    
    if not token:
        raise Exception("unable to retrieve a session token")

    return token

### handle sensitive credentials appropriately

DO NOT STORE YOUR PASSWORD IN PLAIN TEXT on the internet!!<br>

Instead use the Abstract Account object in Domo to store credentials.  They can only be seen in clear text in DomoJupyter and use the same encryption security platform as Domo connectors.

1. create an abstract account object (Data > Accounts).
2. store your credentials as a properly formatted json object<br>
    { "DOMO_USERNAME": "<your_username>", "DOMO_PASSWORD": "<your_password>", "DOMO_INSTANCE": "<domo_instance>"}<br><br>
3. edit this workspace to share the account with this notebook

In [4]:
import domojupyter as domo
# Requires domojupyter Python version 1.0.3

# we will refactor the default script into a function
# account_properties = domo.get_account_property_keys('username_password_auth')
# account_property_value = domo.get_account_property_value('jw_username_password_auth', account_properties[0])


def get_account_credentials(account_name):
    """version 1 of get_account credentaials"""
    
    account_properties = domo.get_account_property_keys(account_name)
    return {prop: domo.get_account_property_value(account_name, prop) for prop in account_properties} # in python this technique is called a "list comprehension" its similar to a for loop

creds = get_account_credentials("username_password_auth")

# umncomment to print your creds, and then recomment so that you don't save your file with your creds in clear text
# creds 

DEVELOPER TIPS
you could have two functions, one for handling any account type, and one specifically for handling abstract credentials.

in the case of abstract credentials, we only want the "credentials" key from the JSON created (notice the modified return statement)

we'll pass an is_abstract_account boolean parameter with a default value of True (assumes we are usually

In [5]:
import json

# to accommod
def get_account_credentials(account_name, is_abstract_account: bool = True):
    """version 2 of get_account credentaials"""
    
    account_properties = domo.get_account_property_keys(account_name)
    res = {prop: domo.get_account_property_value(account_name, prop) for prop in account_properties}
    return json.loads(res['credentials'])

creds = get_account_credentials("username_password_auth")
# creds

## put it all together
1. retrieve username and password from an account object
2. get a session token from get_full_auth
3. authenticate our API request

In [6]:
def get_accounts(domo_instance, session_token):
    # v2 refactored to handle authentication
    
    headers = {"x-domo-authentication": session_token}
    
    url = f'https://{domo_instance}.domo.com/api/data/v2/datasources/providers'
    
    return requests.request(method = "GET",
                            url = url,
                            headers = headers
                           )

In [7]:
creds = get_account_credentials('username_password_auth')

domo_instance = creds['DOMO_INSTANCE']

session_token = get_full_auth(domo_username = creds['DOMO_USERNAME'],
                              domo_password = creds['DOMO_PASSWORD'],
                              domo_instance = creds['DOMO_INSTANCE'] )

get_accounts(domo_instance = domo_instance, session_token = session_token)


<Response [200]>